This notebook plots the output of the script:
`python run_kpsrecoman.py --input-dlmerged [merged_dlreco.root] --input-larflow [larmatchme_larlite.root] -o test_prepsp.root -n 1 -e 0 -tb -mc --stop-after-subclustering`

Running with the `--stop-after-subclustering` halts the reco chain after the spacepoints have been clustered into subcluster fragments for both shower and tracks. These will be used by later steps to form particle clusters.

This notebook takes in:
 * the larmatch output larlite file that stores the output of the larmatch network
 * the keypoint output larlite file that stores the output of run_keypointreco.py
 * optionally, if the mcinfo or dlmerged files is provided that contains truth meta data, then one can also plot the true trajectories of particles

In [ ]:
from __future__ import print_function
import os,sys
import chart_studio as cs
import chart_studio.plotly as py
import plotly.graph_objects as go
import numpy as np
import torch
import ROOT as rt
from larlite import larlite
from ROOT import larutil
import lardly

In [ ]:
# SETUP THE GEOMETRY

# Set the detector in larlite
detid = larlite.geo.kICARUS
#detid = larlite.geo.kMicroBooNE
#detid = larlite.geo.kSBND
larutil.LArUtilConfig.SetDetector(detid)

# Get geometry class
geo = larlite.larutil.Geometry.GetME()

# Get detector outlines from lardly
from lardly.detectors.getdetectoroutlines import DetectorOutlineFromLarlite
detoutline = DetectorOutlineFromLarlite(detid)
detlines = detoutline.getlines()

In [ ]:
PLOT_MC = False
PLOT_KEYPOINTS = True

In [ ]:
# file lists

# The input file
kpreco_output_larlite_v = ["../test_subclustering_larlite.root"]


In [ ]:
io = larlite.storage_manager( larlite.storage_manager.kREAD )
for f in kpreco_output_larlite_v:
    io.add_in_filename( f )
io.open()

In [ ]:
# Get entry and relevant data
ENTRY = 0
io.go_to(ENTRY)

# Get 3D graphs for MC tracks
if PLOT_MC:
    # Get MC info
    mcinfoplots = lardly.data.visualize_nu_interaction(io, do_sce_correction=True )


In [ ]:
# Prepare data for shower keypoints

# define some colors for particle types
kp_color_array = np.array( ((255,0,0),    # nu
                            (0,255,0),    # track-start
                            (0,0,255),    # track-end
                            (255,0,255),  # shower
                            (0,255,255),  # shower-michel
                            (255,255,0)),     # shower-delta
                            dtype=np.float64 ) 

ev_keypoints = io.get_data( larlite.data.kLArFlow3DHit, "keypoint" )
npts = ev_keypoints.size()
print("Number of Reco Keypoints: ",npts)
kp_counts = [0,0,0,0,0,0]
for i in range(npts):
    kphit = ev_keypoints.at(i)
    # data in the hit
    # [0-2]: x,y,z
    # [3]: keypoint type
    # [4]: score
    kptype = int(kphit[3])
    kp_counts[kptype] += 1
print("Counts: ",kp_counts)
    
kp_data = {}
for kptype in range(6):
    if kp_counts[kptype]==0:
        continue
    xyz = np.zeros( (kp_counts[kptype], 3), np.float32 )
    kp_data[kptype] = xyz
    
kp_counts = [0,0,0,0,0,0]

for i in range(npts):
    kphit = ev_keypoints.at(i)
    kptype = int(kphit[3])
    xyz = kp_data[kptype]
    for v in range(3):
        xyz[kp_counts[kptype],v] = kphit[v]
    kp_counts[kptype] += 1

keypoint_traces = []    
for kptype in [5,4,3,2,1,0]:
    if kptype not in kp_data:
        continue
    rgb = kp_color_array[kptype]
    xyz = kp_data[kptype]
    zrgb = "rgb(%d,%d,%d)"%(rgb[0],rgb[1],rgb[2])
    plot_trace = {
        "type":"scatter3d",
        "x": xyz[:,0],
        "y": xyz[:,1],
        "z": xyz[:,2],
        "mode":"markers",
        "name":"KP[%d]"%(kptype),
        "marker":{"color":zrgb,"size":5,"opacity":0.8},
    }
    #print(plot_trace)
    if kptype>=3:
        keypoint_traces += [plot_trace]


In [ ]:
# Plot shower clusters
ev_intime_shower_clusters_v = io.get_data( larlite.data.kLArFlowCluster, "showerkp" )
ev_intime_shower_pca_v = io.get_data( larlite.data.kPCAxis, "showerkp" )
PLOT_KEYPOINTS = True
print("Num of shower clusters: ",ev_intime_shower_clusters_v.size())

intime_shower_cluster_plots = []
for icluster in range(ev_intime_shower_clusters_v.size()):
    lfcluster = ev_intime_shower_clusters_v.at(icluster)
    clusterplot = lardly.data.visualize_larlite_larflowhits( lfcluster, "SHR[%d]"%(icluster))

    print("shower[%d]: num hits=%d"%(icluster,lfcluster.size()))
    
    rvec = np.random.randint(0,high=254,size=3)
    randcolor = "rgb(%d,%d,%d)"%tuple(rvec)
    
    pcacluster = ev_intime_shower_pca_v.at(icluster)
    pcaplot = lardly.data.visualize_pcaxis( pcacluster, idnum=icluster )
    
    clusterplot["marker"]["color"] = randcolor
    clusterplot["marker"]["size"] = 2.0
    pcaplot["line"]["color"] = randcolor
    pcaplot["line"]["width"] = 2.0

    
    intime_shower_cluster_plots.append( clusterplot )
    intime_shower_cluster_plots.append( pcaplot )

intime_shower_cluster_plots += detlines

if PLOT_MC:
    # Get MC info"
    mcinfoplots = lardly.data.visualize_nu_interaction(io, do_sce_correction=True )
    intime_shower_cluster_plots += mcinfoplots
    
if PLOT_KEYPOINTS:
    intime_shower_cluster_plots += keypoint_traces

# hits not part of the clustering
ev_unused_hits = io.get_data( larlite.data.kLArFlow3DHit, "showerkpunused" )
ev_unused_plot = lardly.data.visualize_larlite_larflowhits( ev_unused_hits, "unused", score_threshold=0.0)
ev_unused_plot["marker"]["color"] = "rgb(0,0,0)"
intime_shower_cluster_plots += [ev_unused_plot]

# LAYOUT
axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='In-time track clusters',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": -2, "y": 0.25, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=intime_shower_cluster_plots, layout=layout)
fig.show()

In [ ]:
# Plot in-time track cluster PC axis

ev_intime_track_pca_v = io.get_data( larlite.data.kPCAxis, "trackprojsplit_wcfilter" )
intime_track_pca_plots = []
for icluster in range(ev_intime_track_pca_v.size()):
    lfcluster = ev_intime_track_pca_v.at(icluster)
    clusterplot = lardly.data.visualize_pcaxis( lfcluster, idnum=icluster )
    intime_track_pca_plots.append( clusterplot )

intime_track_pca_plots += detlines

if PLOT_MC:
    # Get MC info"
    mcinfoplots = lardly.data.visualize_nu_interaction(io, do_sce_correction=True )
    #intime_track_pca_plots += mcinfoplots
    
if PLOT_KEYPOINTS:
    intime_track_pca_plots += keypoint_traces

    
# hits not part of the clustering
ev_unused_hits = io.get_data( larlite.data.kLArFlow3DHit, "projsplitnoise" )
ev_unused_plot = lardly.data.visualize_larlite_larflowhits( ev_unused_hits, "projsplitnoise", score_threshold=0.0)
ev_unused_plot["marker"]["color"] = "rgb(0,0,0)"
#intime_track_pca_plots += [ev_unused_plot]

# vetoed hits
ev_vetoed_hits = io.get_data( larlite.data.kLArFlow3DHit, "projsplitvetoed" )
ev_vetoed_plot = lardly.data.visualize_larlite_larflowhits( ev_vetoed_hits, "projsplitvetoed", score_threshold=0.0)
ev_vetoed_plot["marker"]["color"] = "rgb(200,0,0)"
intime_track_pca_plots += [ev_vetoed_plot]
print("type: ",type(intime_track_pca_plots))

# LAYOUT
axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='In-time track cluster PC axis',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": -2, "y": 0.25, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=intime_track_pca_plots, layout=layout)
fig.show()

In [ ]:
# Plot individual in-time track cluster PC axis+cluster

ev_intime_track_pca_v = io.get_data( larlite.data.kPCAxis, "trackprojsplit_wcfilter" )
intime_track_pca_plots = []
print("NPCA: ",ev_intime_track_pca_v.size())
print("NClusters: ",ev_intime_track_clusters_v.size())
for icluster in [10]:
    pcacluster = ev_intime_track_pca_v.at(icluster)
    lfcluster  = ev_intime_track_clusters_v.at(icluster)
    clusterplot = lardly.data.visualize_larlite_larflowhits( lfcluster, "ITRK[%d]"%(icluster), score_threshold=0.5)
    pcaplot = lardly.data.visualize_pcaxis( pcacluster, idnum=icluster )
    
    intime_track_pca_plots.append( clusterplot )
    intime_track_pca_plots.append( pcaplot )

intime_track_pca_plots += detlines

if PLOT_MC:
    # Get MC info"
    mcinfoplots = lardly.data.visualize_nu_interaction(io, do_sce_correction=True )
    #intime_track_pca_plots += mcinfoplots
    
if PLOT_KEYPOINTS:
    intime_track_pca_plots += keypoint_traces

    
# hits not part of the clustering
ev_unused_hits = io.get_data( larlite.data.kLArFlow3DHit, "projsplitnoise" )
ev_unused_plot = lardly.data.visualize_larlite_larflowhits( ev_unused_hits, "projsplitnoise", score_threshold=0.0)
ev_unused_plot["marker"]["color"] = "rgb(0,0,0)"
#intime_track_pca_plots += [ev_unused_plot]

# vetoed hits
ev_vetoed_hits = io.get_data( larlite.data.kLArFlow3DHit, "projsplitvetoed" )
ev_vetoed_plot = lardly.data.visualize_larlite_larflowhits( ev_vetoed_hits, "projsplitvetoed", score_threshold=0.0)
ev_vetoed_plot["marker"]["color"] = "rgb(200,0,0)"
#intime_track_pca_plots += [ev_vetoed_plot]
print("type: ",type(intime_track_pca_plots))

# LAYOUT
axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='In-time track cluster PC axis',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": -2, "y": 0.25, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=intime_track_pca_plots, layout=layout)
fig.show()